In [ ]:
import os
import subprocess

from collections import OrderedDict
from operator import or_
from pprint import pprint
from functools import reduce

In [ ]:
import numpy as np
import pandas as pd
import ujson
import yaml
import yamlloader

In [ ]:
from conda.cli.python_api import run_command, Commands
from conda.history import History

In [ ]:
from conda_env import PY2_PACKAGES

In [ ]:
def get_all_conda_envs():
    '''return all conda environment paths excluding base/root

    using conda's Python API
    '''
    return [
        env
        for env in ujson.loads(run_command(Commands.INFO, '--json')[0])['envs']
        if '/envs/' in env # filter out the root/base env
    ]

In [ ]:
def conda_list(env):
    '''return a DataFrame representation of ``conda list`` for environment at `env`

    this one uses the conda Python API that excludes pip installed packages
    '''
    temp = ujson.loads(run_command(Commands.LIST, '--prefix', env, '--json')[0])
    df = pd.DataFrame(temp)
    df.set_index('name', inplace=True)
    return df

In [ ]:
def conda_list_subprocess(env):
    '''return a DataFrame representation of ``conda list`` for environment at `env`

    this one uses the cli directly that includes pip installed packages
    '''
    temp = ujson.loads(subprocess.run(('conda', 'list', '--prefix', env, '--json'), stdout=subprocess.PIPE).stdout)
    df = pd.DataFrame(temp)
    df.set_index('name', inplace=True)
    return df

In [ ]:
def get_user_installed_packages(env):
    '''return user installed packages in prefix `env`

    using undocumented conda Python API, see
    https://github.com/conda/conda/issues/4545#issuecomment-469984684
    '''
    history = History(env)
    return history.get_requested_specs_map().keys()

In [ ]:
def filter_channels(env, channels=('pypi',)):
    '''return packages from `channels` in environment `env`
    '''
    df = conda_list_subprocess(env)
    return df[df.channel.isin(channels)].index

In [ ]:
def remove_element(list_, elements):
    '''filter out items in `list_` that contains element in `elements`
    '''
    return [
        item for item in list_
        if not reduce(
            or_,
            (element in item for element in elements)
        )
    ]

In [ ]:
def map_union(func, iterables):
    '''set union of the results from `func` applied to items in `iterables`
    '''
    return set().union(*[set(func(item)) for item in iterables])

In [ ]:
envs = remove_element(get_all_conda_envs(), ('jupyterlab', 'ISR', 'sage', 'data100', 'acx'))

List of environments that will be inspected:

In [ ]:
envs

# Conda

In [ ]:
# set of all user installed packages in envs
conda_packages = map_union(get_user_installed_packages, envs)

In [ ]:
with open(os.path.expanduser('conda-all.txt'), 'r') as f:
    conda_all = set(word.lstrip('#').strip() for word in f.readlines() if word.lstrip('#').strip())

In [ ]:
with open(os.path.expanduser('conda.txt'), 'r') as f:
    conda = set(word.lstrip('#').strip() for word in f.readlines() if word.lstrip('#').strip())

In [ ]:
conda_all |= conda

In [ ]:
conda_all2 = conda_all | set(PY2_PACKAGES)

User installed packages not in `conda-all.txt` or `conda.txt`

In [ ]:
list(map(print, sorted(conda_packages - conda_all2)));

in `conda-all.txt` or `conda.txt` but not installed

In [ ]:
list(map(print, sorted(conda_all2 - conda_packages)));

# pip

In [ ]:
# all pypi packages from envs
pip_packages = map_union(filter_channels, envs)

In [ ]:
with open(os.path.expanduser('pip-all.txt'), 'r') as f:
    pip_all = set(word.lstrip('#').strip() for word in f.readlines() if word.lstrip('#').strip())

In [ ]:
with open(os.path.expanduser('pip.txt'), 'r') as f:
    pip = set(word.lstrip('#').strip() for word in f.readlines() if word.lstrip('#').strip())

In [ ]:
pip_all |= pip

pypi packages not in `pip-all.txt` or `pip.txt`

In [ ]:
list(map(print, sorted(pip_packages - pip_all)));

in `pip-all.txt` or `pip.txt` but not installed

In [ ]:
list(map(print, sorted(pip_all - pip_packages)));

# Intersection of Anaconda supported packages

Create an environment named `acx`, which stands for Anaconda extended, as an intersection of packages installed and those supported by Anaconda

In [ ]:
df_linux = pd.read_html('https://docs.anaconda.com/anaconda/packages/py3.7_linux-64/', header=0, index_col=0)[0]

In [ ]:
df_mac = pd.read_html('https://docs.anaconda.com/anaconda/packages/py3.7_osx-64/', header=0, index_col=0)[0]

In [ ]:
# The pandas parser didn't pick up the ticks in last column
np.all(df_linux['In Installer'].isna()), np.all(df_mac['In Installer'].isna())

In [ ]:
conda_supported_packages_linux = set(df_linux.index)
conda_supported_packages_mac = set(df_mac.index)
conda_supported_packages = conda_supported_packages_linux | conda_supported_packages_mac

In [ ]:
len(conda_supported_packages_mac), len(conda_supported_packages_linux), len(conda_supported_packages)

In [ ]:
# packages in conda_all.txt or conda.txt, that's supported by Anaconda
conda_filtered_linux = conda_all & conda_supported_packages_linux
conda_filtered_mac = conda_all & conda_supported_packages_mac
conda_filtered = conda_filtered_mac & conda_filtered_linux
len(conda_filtered), len(conda_filtered_mac), len(conda_filtered_linux), conda_filtered_linux - conda_filtered_mac

In [ ]:
conda_filtered.update({'anaconda', 'panflute', 'cytoolz'})

In [ ]:
conda_filtered = sorted(conda_filtered)

In [ ]:
len(conda_filtered)

In [ ]:
with open('acx.yml', 'w') as f:
    yaml.dump(
        OrderedDict((
            ('name', 'acx'),
            ('channels', ['defaults']),
            ('dependencies', conda_filtered)
        )),
        f,
        Dumper=yamlloader.ordereddict.CSafeDumper,
        default_flow_style=False
    )